<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/DeepAR_YJ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install mxnet and gluonts

In [0]:
pip install --upgrade mxnet==1.6 gluonts

     |████████████████████████████████| 68.7MB 59kB/s 
     |████████████████████████████████| 419kB 48.9MB/s 
     |████████████████████████████████| 194kB 49.9MB/s 
     |████████████████████████████████| 7.4MB 20.0MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68018 sha256=77fab4affa0529f746500d005610fd939ce13af6d62fe269af8881f25049e578
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


## Dataset Loading

In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import json
#import os

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
#from gluonts.dataset.util import to_pandas

In [0]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [0]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.distribution.gaussian import GaussianOutput
from gluonts.distribution.student_t import StudentTOutput
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

## Experiment of DeepAR

## electricity

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/test/data.json


In [0]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [0]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=elec_dataset.metadata.freq, 
    prediction_length=elec_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:14<00:00,  3.39it/s, epoch=100/100, avg_epoch_loss=5.12]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 43151.75it/s]


{
    "MSE": 2092254.1436080525,
    "abs_error": 8443244.096513748,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 0.7433946969760163,
    "MAPE": 0.09550073439391775,
    "sMAPE": 0.10736025187924114,
    "OWA": NaN,
    "MSIS": 9.34018030329882,
    "QuantileLoss[0.1]": 3492565.4570164047,
    "Coverage[0.1]": 0.10033748701972979,
    "QuantileLoss[0.5]": 8443244.150904069,
    "Coverage[0.5]": 0.4498590713543987,
    "QuantileLoss[0.9]": 5374310.903311058,
    "Coverage[0.9]": 0.8490765465064509,
    "RMSE": 1446.4626312518594,
    "NRMSE": 0.6064140886115567,
    "ND": 0.06563826533313709,
    "wQuantileLoss[0.1]": 0.027151404784761416,
    "wQuantileLoss[0.5]": 0.06563826575597057,
    "wQuantileLoss[0.9]": 0.04178020213817568,
    "mean_wQuantileLoss": 0.04485662422630255,
    "MAE_Coverage": 0.03380062305296007
}


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 7461.97it/s]


{
    "MSE": 459631.0071551227,
    "abs_error": 897940.0201396942,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.6542193132332207,
    "MAPE": 0.08108305303708697,
    "sMAPE": 0.1022167482942089,
    "OWA": NaN,
    "MSIS": 7.710735864877391,
    "QuantileLoss[0.1]": 380458.6035026938,
    "Coverage[0.1]": 0.12318276220145379,
    "QuantileLoss[0.5]": 897940.0166884828,
    "Coverage[0.5]": 0.5242731048805811,
    "QuantileLoss[0.9]": 512635.18815480167,
    "Coverage[0.9]": 0.8922637590861894,
    "RMSE": 677.960918604548,
    "NRMSE": 0.30245016065087266,
    "ND": 0.05199723065211571,
    "wQuantileLoss[0.1]": 0.022031308680098435,
    "wQuantileLoss[0.5]": 0.05199723045226557,
    "wQuantileLoss[0.9]": 0.029685290243249313,
    "mean_wQuantileLoss": 0.034571276458537775,
    "MAE_Coverage": 0.018397369331948507
}


## exchange_rate

In [0]:
exchange_rate_dataset = get_dataset("exchange_rate", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/test/data.json


In [0]:
print (len(exchange_rate_dataset.train))
print (len(exchange_rate_dataset.test))

8
40


In [0]:
exchange_rate_dataset.metadata

MetaData(freq='1B', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=30)

In [0]:
exchange_rate_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')

In [0]:
car = [int(exchange_rate_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(exchange_rate_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=exchange_rate_dataset.metadata.freq, 
    prediction_length=exchange_rate_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(exchange_rate_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:07<00:00,  6.33it/s, epoch=100/100, avg_epoch_loss=-4.2]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=exchange_rate_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(exchange_rate_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 40/40 [00:00<00:00, 837.25it/s]


{
    "MSE": 0.0003244642393733936,
    "abs_error": 14.109951864928007,
    "abs_target_sum": 975.9766580164433,
    "abs_target_mean": 0.8133138816803692,
    "seasonal_error": 0.006515919403041841,
    "MASE": 2.916689351928164,
    "MAPE": 0.016858252813219475,
    "sMAPE": 0.016615119229724105,
    "OWA": NaN,
    "MSIS": 40.880627394259236,
    "QuantileLoss[0.1]": 9.054444624297322,
    "Coverage[0.1]": 0.4691666666666666,
    "QuantileLoss[0.5]": 14.109951863996685,
    "Coverage[0.5]": 0.8658333333333333,
    "QuantileLoss[0.9]": 5.212419524043797,
    "Coverage[0.9]": 0.9841666666666669,
    "RMSE": 0.018012890922153323,
    "NRMSE": 0.022147526714947127,
    "ND": 0.01445726365383042,
    "wQuantileLoss[0.1]": 0.009277316777943651,
    "wQuantileLoss[0.5]": 0.014457263652876173,
    "wQuantileLoss[0.9]": 0.005340721503152976,
    "mean_wQuantileLoss": 0.009691767311324267,
    "MAE_Coverage": 0.2730555555555556
}


## m4_daily

In [0]:
m4_daily_dataset = get_dataset("m4_daily", regenerate=True)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


In [0]:
print (len(m4_daily_dataset.train))
print (len(m4_daily_dataset.test))

4227
4227


In [0]:
m4_daily_dataset.metadata

MetaData(freq='D', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=14)

In [0]:
m4_daily_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')

In [0]:
car = [int(m4_daily_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_daily_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_daily_dataset.metadata.freq, 
    prediction_length=m4_daily_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_daily_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:04<00:00, 12.16it/s, epoch=100/100, avg_epoch_loss=5.94]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_daily_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_daily_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 4227/4227 [00:00<00:00, 60252.58it/s]


{
    "MSE": 445438.51894085354,
    "abs_error": 12480753.005447388,
    "abs_target_sum": 383097653.5168457,
    "abs_target_mean": 6473.649895515986,
    "seasonal_error": 52.78075560259988,
    "MASE": 3.895732915826975,
    "MAPE": 0.05162706219952426,
    "sMAPE": 0.03609470133934619,
    "OWA": NaN,
    "MSIS": 61.503349190341865,
    "QuantileLoss[0.1]": 7369882.405699158,
    "Coverage[0.1]": 0.10285917063773696,
    "QuantileLoss[0.5]": 12480753.042640686,
    "Coverage[0.5]": 0.35448984419885876,
    "QuantileLoss[0.9]": 7245212.499118041,
    "Coverage[0.9]": 0.7436716347291222,
    "RMSE": 667.4118061143761,
    "NRMSE": 0.10309667913562379,
    "ND": 0.03257851592374365,
    "wQuantileLoss[0.1]": 0.019237607795410543,
    "wQuantileLoss[0.5]": 0.03257851602082934,
    "wQuantileLoss[0.9]": 0.01891218187479567,
    "mean_wQuantileLoss": 0.023576101897011852,
    "MAE_Coverage": 0.10156589723658532
}


## m4_hourly

In [0]:
m4_hourly_dataset = get_dataset("m4_hourly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/test/data.json


In [0]:
print (len(m4_hourly_dataset.train))
print (len(m4_hourly_dataset.test))

414
414


In [0]:
m4_hourly_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=48)

In [0]:
m4_hourly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')

In [0]:
car = [int(m4_hourly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_hourly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_hourly_dataset.metadata.freq, 
    prediction_length=m4_hourly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_hourly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:08<00:00,  5.82it/s, epoch=100/100, avg_epoch_loss=3.37]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_hourly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_hourly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 414/414 [00:00<00:00, 7788.62it/s]


{
    "MSE": 1483002.7640645003,
    "abs_error": 5739086.591159821,
    "abs_target_sum": 145558863.59960938,
    "abs_target_mean": 7324.822041043147,
    "seasonal_error": 336.9046924038302,
    "MASE": 1.762556630899094,
    "MAPE": 0.12414278600037203,
    "sMAPE": 0.11125520445161138,
    "OWA": NaN,
    "MSIS": 21.53143925333888,
    "QuantileLoss[0.1]": 2584647.0212579733,
    "Coverage[0.1]": 0.19051932367149754,
    "QuantileLoss[0.5]": 5739086.530657768,
    "Coverage[0.5]": 0.5902274557165861,
    "QuantileLoss[0.9]": 3246079.373098373,
    "Coverage[0.9]": 0.8478764090177134,
    "RMSE": 1217.7860091430268,
    "NRMSE": 0.1662546888264877,
    "ND": 0.039427943096247295,
    "wQuantileLoss[0.1]": 0.017756713382756237,
    "wQuantileLoss[0.5]": 0.03942794268059379,
    "wQuantileLoss[0.9]": 0.022300801839367234,
    "mean_wQuantileLoss": 0.026495152634239086,
    "MAE_Coverage": 0.0776234567901234
}


## m4_monthly

In [0]:
m4_monthly_dataset = get_dataset("m4_monthly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


In [0]:
print (len(m4_monthly_dataset.train))
print (len(m4_monthly_dataset.test))

48000
48000


In [0]:
m4_monthly_dataset.metadata

MetaData(freq='M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=18)

In [0]:
m4_monthly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')

In [0]:
car = [int(m4_monthly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_monthly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_monthly_dataset.metadata.freq, 
    prediction_length=m4_monthly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_monthly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:04<00:00, 11.99it/s, epoch=76/100, avg_epoch_loss=5.98]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_monthly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_monthly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 48000/48000 [00:00<00:00, 286184.63it/s]


{
    "MSE": 2115003.670783878,
    "abs_error": 565092582.0651245,
    "abs_target_sum": 4157157295.381714,
    "abs_target_mean": 4811.5246474325495,
    "seasonal_error": 616.9975625355751,
    "MASE": 1.3675626003803907,
    "MAPE": 0.1775026495247414,
    "sMAPE": 0.14946500711249788,
    "OWA": NaN,
    "MSIS": 32.11634113345029,
    "QuantileLoss[0.1]": 408859534.2665299,
    "Coverage[0.1]": 0.29263541666664467,
    "QuantileLoss[0.5]": 565092582.143055,
    "Coverage[0.5]": 0.4336388888888755,
    "QuantileLoss[0.9]": 496985676.9068374,
    "Coverage[0.9]": 0.6354907407407671,
    "RMSE": 1454.3052192658452,
    "NRMSE": 0.30225455044522503,
    "ND": 0.13593245141166524,
    "wQuantileLoss[0.1]": 0.09835074913348643,
    "wQuantileLoss[0.5]": 0.13593245143041133,
    "wQuantileLoss[0.9]": 0.1195494039782788,
    "mean_wQuantileLoss": 0.11794420151405886,
    "MAE_Coverage": 0.17450192901233405
}


## m4_quarterly

In [0]:
m4_quarterly_dataset = get_dataset("m4_quarterly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/test/data.json


In [0]:
print (len(m4_quarterly_dataset.train))
print (len(m4_quarterly_dataset.test))

24000
24000


In [0]:
m4_quarterly_dataset.metadata

MetaData(freq='3M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=8)

In [0]:
m4_quarterly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')

In [0]:
car = [int(m4_quarterly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_quarterly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_quarterly_dataset.metadata.freq, 
    prediction_length=m4_quarterly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_quarterly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 22.03it/s, epoch=100/100, avg_epoch_loss=6.87]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_quarterly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_quarterly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 24000/24000 [00:00<00:00, 254270.87it/s]


{
    "MSE": 1972884.9687406262,
    "abs_error": 117241708.11991882,
    "abs_target_sum": 1147074070.9031982,
    "abs_target_mean": 5974.344119287491,
    "seasonal_error": 473.4332698179725,
    "MASE": 1.3550683143872055,
    "MAPE": 0.13164692057705824,
    "sMAPE": 0.10918382057315212,
    "OWA": NaN,
    "MSIS": 18.527260001250795,
    "QuantileLoss[0.1]": 71331246.12868805,
    "Coverage[0.1]": 0.167265625,
    "QuantileLoss[0.5]": 117241708.0598526,
    "Coverage[0.5]": 0.5809583333333334,
    "QuantileLoss[0.9]": 74032826.25184327,
    "Coverage[0.9]": 0.8765520833333333,
    "RMSE": 1404.5942363332645,
    "NRMSE": 0.23510434087629664,
    "ND": 0.10220936127307237,
    "wQuantileLoss[0.1]": 0.062185387969342136,
    "wQuantileLoss[0.5]": 0.10220936122070765,
    "wQuantileLoss[0.9]": 0.0645405803598632,
    "mean_wQuantileLoss": 0.07631177651663766,
    "MAE_Coverage": 0.057223958333333345
}


## m4_weekly

In [0]:
m4_weekly_dataset = get_dataset("m4_weekly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/test/data.json


In [0]:
print (len(m4_weekly_dataset.train))
print (len(m4_weekly_dataset.test))

359
359


In [0]:
m4_weekly_dataset.metadata

MetaData(freq='W', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=13)

In [0]:
m4_weekly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')

In [0]:
car = [int(m4_weekly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_weekly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_weekly_dataset.metadata.freq, 
    prediction_length=m4_weekly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_weekly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:03<00:00, 15.96it/s, epoch=100/100, avg_epoch_loss=6.01]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_weekly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_weekly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 359/359 [00:00<00:00, 5887.35it/s]


{
    "MSE": 461244.35867754853,
    "abs_error": 1699570.7895736694,
    "abs_target_sum": 25616889.502441406,
    "abs_target_mean": 5488.941397566194,
    "seasonal_error": 216.54813922503882,
    "MASE": 3.1596356283777807,
    "MAPE": 0.08607867269158756,
    "sMAPE": 0.08943152718725707,
    "OWA": NaN,
    "MSIS": 40.13492853924365,
    "QuantileLoss[0.1]": 773217.9935211182,
    "Coverage[0.1]": 0.07713734733233336,
    "QuantileLoss[0.5]": 1699570.7870101929,
    "Coverage[0.5]": 0.3582601242768377,
    "QuantileLoss[0.9]": 848180.0680007933,
    "Coverage[0.9]": 0.8247268052281977,
    "RMSE": 679.1497321486246,
    "NRMSE": 0.12373054892693167,
    "ND": 0.06634571263664515,
    "wQuantileLoss[0.1]": 0.030183914149585843,
    "wQuantileLoss[0.5]": 0.06634571253657537,
    "wQuantileLoss[0.9]": 0.033110189584881405,
    "mean_wQuantileLoss": 0.04321327209034754,
    "MAE_Coverage": 0.07995857438754374
}


## m4_yearly

In [0]:
m4_yearly_dataset = get_dataset("m4_yearly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/test/data.json


In [0]:
print (len(m4_yearly_dataset.train))
print (len(m4_yearly_dataset.test))

23000
23000


In [0]:
m4_yearly_dataset.metadata

MetaData(freq='12M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=6)

In [0]:
m4_yearly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')

In [0]:
car = [int(m4_yearly_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(m4_yearly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_yearly_dataset.metadata.freq, 
    prediction_length=m4_yearly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(m4_yearly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:01<00:00, 27.13it/s, epoch=81/100, avg_epoch_loss=7.66]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_yearly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_yearly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 23000/23000 [00:00<00:00, 186059.37it/s]


{
    "MSE": 3592698.3408726887,
    "abs_error": 128628600.16764832,
    "abs_target_sum": 860724679.6655273,
    "abs_target_mean": 6237.135359895143,
    "seasonal_error": 288.79964609563865,
    "MASE": 3.957290589332747,
    "MAPE": 0.19593046713750686,
    "sMAPE": 0.15689448032743547,
    "OWA": NaN,
    "MSIS": 62.54054492637989,
    "QuantileLoss[0.1]": 64984800.1826233,
    "Coverage[0.1]": 0.23689130434782835,
    "QuantileLoss[0.5]": 128628600.15127563,
    "Coverage[0.5]": 0.5956086956521757,
    "QuantileLoss[0.9]": 96877857.22847137,
    "Coverage[0.9]": 0.8115652173913072,
    "RMSE": 1895.4414633200067,
    "NRMSE": 0.3038961564803802,
    "ND": 0.14944221213412012,
    "wQuantileLoss[0.1]": 0.07550010092411434,
    "wQuantileLoss[0.5]": 0.14944221211509814,
    "wQuantileLoss[0.9]": 0.11255382762594601,
    "mean_wQuantileLoss": 0.11249871355505282,
    "MAE_Coverage": 0.10697826086956563
}


## traffic

In [0]:
traffic_dataset = get_dataset("traffic", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/test/data.json


In [0]:
print (len(traffic_dataset.train))
print (len(traffic_dataset.test))

862
6034


In [0]:
traffic_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [0]:
traffic_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')

In [0]:
car = [int(traffic_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(traffic_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=traffic_dataset.metadata.freq, 
    prediction_length=traffic_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(traffic_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:11<00:00,  4.36it/s, epoch=100/100, avg_epoch_loss=-3.95]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=traffic_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(traffic_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 6034/6034 [00:00<00:00, 68520.80it/s]


{
    "MSE": 0.000576183923502055,
    "abs_error": 1338.9888566527516,
    "abs_target_sum": 8672.5710073933,
    "abs_target_mean": 0.05988682885450042,
    "seasonal_error": 0.015220711169889612,
    "MASE": 0.5944441305347573,
    "MAPE": 0.22375201176215057,
    "sMAPE": 0.16246708424071443,
    "OWA": NaN,
    "MSIS": 8.399648532411451,
    "QuantileLoss[0.1]": 592.308628594046,
    "Coverage[0.1]": 0.11006380510440854,
    "QuantileLoss[0.5]": 1338.9888563205768,
    "Coverage[0.5]": 0.6581938459838709,
    "QuantileLoss[0.9]": 926.0256117628236,
    "Coverage[0.9]": 0.9301803668102988,
    "RMSE": 0.024003831433795208,
    "NRMSE": 0.40081987797541146,
    "ND": 0.1543935305356709,
    "wQuantileLoss[0.1]": 0.068296774749853,
    "wQuantileLoss[0.5]": 0.15439353049736912,
    "wQuantileLoss[0.9]": 0.10677636550607586,
    "mean_wQuantileLoss": 0.10982222358443265,
    "MAE_Coverage": 0.06614600596619273
}


## electricity2

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

In [0]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

In [0]:
elec_dataset.metadata

In [0]:
elec_dataset.metadata.feat_static_cat[0]

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [0]:
next(iter(elec_dataset.train)).keys()

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=elec_dataset.metadata.freq, 
    prediction_length=elec_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [0]:
DA_predictor = DA_estimator.train(elec_dataset.train)

In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))